<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/part2_searching_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with IDC - Part 2: Basics of searching IDC data

---


## Summary

This notebook is part of [the "Getting started with IDC" notebook series](https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/getting_started) introducing NCI Imaging Data Commons to the users who want to interact with IDC programmatically.

In [Part 1](https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/getting_started/part1_prerequisites.ipynb) of this tutorial series, you learned how to complete the prerequisites that are required to be able to search IDC data using GCP BigQuery.

In this notebook you will be introduced into how IDC organizes the metadata accompanying images available in IDC, and how that metadata can be used to define subsets of data. This documentation page complements this notebook with the more detailed discussion of the metadata organization in IDC: https://learn.canceridc.dev/data/organization-of-data/files-and-metadata.

---
Initial version: Nov 2022

Updated: Oct 2023


## What is IDC?

NCI Imaging Data Commons (IDC) is a cloud-based repository of publicly available cancer imaging data co-located with the analysis and exploration tools and resources. IDC is a node within the broader NCI Cancer Research Data Commons (CRDC) infrastructure that provides secure access to a large, comprehensive, and expanding collection of cancer research data.

## Prerequisites

It is assumed that you completed tutorial 1 in the series. You will need to have a Google identity, Google Cloud activated for that account, and the ID of the project created under that account.

Before doing **anything** you **must**  authorize Colab Runtime to act on your behalf, and initialize the variable pointing to your Google Cloud project ID.

The following cell initializes project ID that is needed for all operations with the cloud. You should have project ID if you completed the [first tutorial](https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/getting_started/part1_prerequisites.ipynb) in this series.

In [1]:
#@title Enter your Project ID
# initialize this variable with your Google Cloud Project ID!
#this project_id is injected for testing
my_ProjectID='idc-external-025'

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

from google.colab import auth
#while testing, the authentication is handled by using application default credentials
#auth.authenticate_user()

## Why do I need to search?

Think of IDC as a library. Image files are books, and we have ~45 TB of those. When you go to a library, you want to check out just the books that you want to read. In order to find a book in a large library you need a catalog.

Just as in the library, IDC maintains a catalog that indexes a variety of metadata fields describing the files we curate. That metadata catalog is accessible in a large database table that you should be using to search and subset the images. Each row in that table corresponds to a file, and includes the location of the file alongside the metadata attributes describing that file.


## How do I search?

When you search, or _query_ IDC catalog, you specify what criteria should the metadata describing the selected files satisfy.

Queries can be as simple as

* "_everything in collection X_",

or as complex as

* "_files corresponding to CT images of female patients that are accompanied by annotations of lung tumors that are larger than 1500 mm^3 in volume_".

Although it would be very nice to just state what you need in free form, in practice queries need to be written in a formal way.

IDC organizes all of the metadata into large tables, where each row corresponds to one image file (as of IDC data release v12, we index ~42 millions of files) and each column represents a metadata attribute present in one or more files in IDC (currently, we index hundreds of such attributes).

IDC metadata tables are maintained in [GCP BigQuery](https://cloud.google.com/bigquery),  with only a tiny subset of the attributes indexed in the catalog available via the [IDC Portal exploration page](https://imaging.datacommons.cancer.gov/explore/). IDC metadata can be queried using Standard Query Language (SQL), and does not require learning any IDC-specific API.

In the following steps of the tutorial we will use just a few of the attributes (SQL table columns) to get started. You will be able to use the same principles and SQL queries to extend your search criteria to include any of the other attributes indexed by IDC.

## First query and BigQuery workspace

To get started, let's build the queries that replicate some of the information about IDC data shown in the IDC Portal.

As the very first query, let's get the list of all the image collections available in IDC. Here is that query:

```sql
SELECT
  DISTINCT(collection_id)
FROM
  bigquery-public-data.idc_current.dicom_all
```

To run this query interactively, copy the query above to the clipboard, paste it into the query tab in the [BigQuery SQL workspace](https://console.cloud.google.com/bigquery), and hit the "Run" button. Within few moments you should be able to see the list of collections in IDC in the "Query results" section of the interface.

![bq_run](https://www.dropbox.com/s/6ah98n6e9ik18if/bq_run.png?raw=1)

Let's look into how this query works:
* `SELECT` defines the list of columns that should be returned by the query,
* `DISTINCT` indicates that we want to see the distinct values encountered in the selected column,
* `FROM` defines which table should be queried.

In this query we work with the [`dicom_all` table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table), which contains the DICOM metadata extracted from IDC images along with collection-level metadata that does not originate from DICOM.

## Organization of IDC metadata in BigQuery tables

Let's take a moment to look into the table used in the `FROM` clause of our query: [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table).

This name is like an address that allows to locate the specific table in BigQuery. This "address" consists of three components: <project_id>.<dataset_id>.<table_id>

1. `bigquery-public-data` is the ID of a public GCP _project_ that is maintained by Google Public Datasets Program. IDC-curated BigQuery tables with the metadata about IDC images are included in this project.
2. `idc_current`is a _dataset_ within the `bigquery-public-data` project. Think of BigQuery datasets as containers that are used to organize and control access to the tables within the project.
3. `dicom_all` is one of the tables within the `idc_current` dataset. As you spend more time learning about IDC, you will hopefully leverage other tables available in that dataset.

If you now look back at the [BigQuery console](https://console.cloud.google.com/bigquery) and expand the list of datasets under the `bigquery-public-data` project, you will see that in addition to the `idc_current` dataset there are also datasets `idc_v14`, `idc_v13`, etc all the way to `idc_v1`. Those datasets correspond to the IDC data release versions, with `idc_current` being an alias for the latest (at the moment of writing this, v14 is the latest release) version of IDC data.

We will not spend time discussing how IDC versioning works, but it is important to know that

1. IDC data is versioned;
2. queries against the `idc_current` dataset are equivalent to the queries against the latest version (currently, `idc_v14`) of IDC data;
3. if you want the results of the queries to be persistent, write those against `idc_v*` datasets instead of `idc_current`.

## Same query using Python SDK

BigQuery SQL workspace is a very convenient tool for exploring schemas of the tables, experimenting with writing and debugging queries, profiling their execution. But you can also run those queries programmatically, which is very convenient if you want to direct the result the query into a pandas dataframe, or just perform your searches programmatically.

BigQuery API support is implemented in a variety of languages, with the python bindings available in the `google-cloud-bigquery` package. Conveniently, this package is pre-installed in Colab!

**HINT**: SQL query syntax is not sensitive to indentation or capitalization - although those are quite helpful to make the query more readable!

In [2]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT
  DISTINCT(collection_id)
FROM
  bigquery-public-data.idc_current.dicom_all
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,collection_id
0,cmb_lca
1,midrc_ricord_1b
2,icdc_glioma
3,pancreas_ct
4,prostate_3t
...,...
132,cptac_lscc
133,rider_breast_mri
134,qin_breast_dce_mri
135,prostate_mri_us_biopsy


## Exploring other IDC portal attributes via SQL

Next we will explore few other attributes that are available in the [IDC portal](https://imaging.datacommons.cancer.gov/) (with the few exceptions, the mapping is pretty intuitive):

![portal_filters](https://www.dropbox.com/s/qt3dhzara1ap7s3/portal_filters.png?raw=1)

* "Primary site location" maps to the `collection_tumorLocation` column
* "Body Part Examined" -> `BodyPartExamined`
* "Modality" -> `Modality`



### Exercise 1: Write the query to list all values for `collection_tumorLocation`

In the cell below, we pre-filled the code by copying the previous cell. All you need to do is write the query!

As always, you can use the the Editor tab in the [BigQuery SQL workspace](https://console.cloud.google.com/bigquery) to experiment and debug the query if you prefer!

In [3]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

# Execution of this cell will fail unless you wrote the query below!
selection_query = """
#this query is injected for testing
 SELECT DISTINCT collection_tumorLocation FROM `bigquery-public-data.idc_current.dicom_all` 
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,collection_tumorLocation
0,Ovary
1,Adrenal
2,Skin
3,Ear
4,Prostate
5,"Abdomen, Pelvis"
6,Head-Neck
7,Thyroid
8,Adrenal Glands
9,Lung Phantom


## Combining multiple attributes in a single query

With the understanding of the attributes that are available in IDC, you can use SQL to build queries that summarize those attributes to better understand the data.

In the following, we use the earlier discussed attributes to list distinct values encountered for tumor locations, body parts examined and modalities for the individual collections. We will also use the attribute that captures the species covered by the collection.

In [4]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT
  collection_id,
  STRING_AGG(DISTINCT(Modality)) as modalities,
  STRING_AGG(DISTINCT(collection_tumorLocation)) as tumor_locations,
  STRING_AGG(DISTINCT(collection_species)) as species
FROM
  bigquery-public-data.idc_current.dicom_all
GROUP BY
  collection_id
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,collection_id,modalities,tumor_locations,species
0,qin_prostate_repeatability,"SR,SEG,MR",Prostate,Human
1,nlm_visible_human_project,"XC,CT,MR",Whole Body,Human
2,cmb_mel,"US,CT,PT",Various,Human
3,lctsc,"RTSTRUCT,CT",Lung,Human
4,tcga_chol,SM,Bile Duct,Human
...,...,...,...,...
132,breast_diagnosis,"CT,MG,MR,SR,PT",Breast,Human
133,cptac_coad,SM,Colon,Human
134,cmb_gec,"PT,CT",Esophagus,Human
135,tcga_tgct,SM,Testicles,Human


In the query above, we use the familiar operators `SELECT` and `FROM`, but also couple of new ones:

* `GROUP BY` in the end of the query indicates that we want to get a single row per the distinct value of the `collection_id`
* `STRING_AGG` and `DISTINCT` indicate how the values of the selected columns should be aggregated while combining into single row per `collection_id`: we take all the distinct values per individual `collection_id`, and the concatenate them into a single string



### Exercise 2: Use multiple criteria to select image collections of interest

In the previous query we learned how to get collection-level list of the modalities and tumor locations. In this exercise you task is to write a query that identifies all of the collections that include MR modality and "Lung" as a tumor location.

As always, you can use the the Editor tab in the [BigQuery SQL workspace](https://console.cloud.google.com/bigquery) to experiment and debug the query if you prefer!

In [5]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

# Execution of this cell will fail unless you wrote the query below!
selection_query = """
SELECT
  DISTINCT(collection_id)
FROM
  bigquery-public-data.idc_current.dicom_all
WHERE
  #this query is injected for testing
 MODALITY='MR' AND collection_tumorLocation='Lung'
  # Use AND operator to combine the filter values for the
  # Modality and collection_tumorLocation to select collections that
  # include MR images for Lung cancer locations
  # Note that SQL uses single = for comparison, and strings should
  # be enclosed in ""
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,collection_id
0,acrin_nsclc_fdg_pet
1,cptac_luad
2,covid_19_ny_sbu
3,cmb_lca


## DICOM data model: Patients, studies, series and instances

Up to now we searched the data at the granularity of the collections. In practice, we often want to know how many patients meet our search criteria, or what are the specific images that we need to download.

IDC is using DICOM for data representation, and in the DICOM data model, patients (identified by `PatientID`) undergo imaging exams (or _studies_, in DICOM nomenclature).

Each patient will have one or more studies, with each study identified uniquely by the attribute `StudyInstanceUID`. During each of the imaging studies one or more imaging _series_ will be collected. As an example, a Computed Tomography (CT) imaging study may include a volume sweep before and after administration of the contrast agent. Imaging series are uniqiely identified by `SeriesInstanceUID`.

Finally, each imaging series contains one or more _instances_, where each instance corresponds to a file. Most often, one instance corresponds to a single slice from a cross-sectional image. Individual instances are identified by unique `SOPInstanceUID` values.

The figure below, borrowed from the DICOM standard [here](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/chapter_7.html), captures the discussed data model.

![DICOM data model](https://2103490465-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-MCTG4fXybYgGMalZnmf-2668963341%2Fuploads%2Fgit-blob-0f639d56e22ae53a03c2ca59c96306c5db51b158%2FPS3.3_7-1a-DICOM_model.png?alt=media)

Let's go over some examples of how those concepts can help us navigate data in IDC. Here is how we would count all the patients that have MR (Magnetic Resonance) images:

In [6]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT
  COUNT(DISTINCT(PatientID)) as patient_cnt
FROM
  bigquery-public-data.idc_current.dicom_all
WHERE
  Modality = "MR"
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,patient_cnt
0,6654


### Exercise 3: Learn more about the size of the collections in IDC

In this exercise your task is to expand on the query we wrote earlier to calculate additional characteristics of the collections by counting the number of distinct values of `PatientID`, `StudyInstanceUID`, `SeriesInstanceUID` and `SOPInstanceUID` in each of the collections.

As always, you can use the the Editor tab in the [BigQuery SQL workspace](https://console.cloud.google.com/bigquery) to experiment and debug the query if you prefer!

In [7]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

# Execution of this cell will fail unless you wrote the query below!
selection_query = """
SELECT
  collection_id,
  STRING_AGG(DISTINCT(Modality)) as modalities,
  STRING_AGG(DISTINCT(collection_tumorLocation)) as tumor_locations,
  STRING_AGG(DISTINCT(collection_species)) as species
  # count number of distinct patients
  # count number of distinct studies
  # count number of distinct series
  # count number of distinct instances
FROM
  bigquery-public-data.idc_current.dicom_all
GROUP BY
  collection_id
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,collection_id,modalities,tumor_locations,species
0,nlm_visible_human_project,"XC,CT,MR",Whole Body,Human
1,cmb_mel,"US,CT,PT",Various,Human
2,cptac_sar,"SM,CT,PT,MR","Abdomen, Arm, Bladder, Chest, Head-Neck, Kidne...",Human
3,tcga_thym,SM,Thymus,Human
4,ct_lymph_nodes,"CT,SEG","Abdomen, Mediastinum",Human
...,...,...,...,...
132,tcga_prad,"PT,SM,MR,CT",Prostate,Human
133,c4kc_kits,"SEG,CT",Kidney,Human
134,cmb_pca,"MR,CT,NM",Prostate,Human
135,pdmr_521955_158_r4,"MR,SR",Abdomen,Mouse


# Selecting series of specific type

In many cases, image analysis is done at the granularity of the individual DICOM series. In some cases DICOM series corresponds to a single instance (e.g., for X-ray modalities), but in most cases imaging modalities are cross-sectional, containing multiple slices, with each slice stored in a separate instance (file), which can be reconstructed into a 3D volume.

From the examples and queries above, you should have developed some understanding about the modalities and few other collection-level characteristics for the data included in IDC. As an example, we know that IDC data contains MR images of Liver.

In the following query we select the UID of a sample MR series from the images covering Liver cancer.

In [8]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT
  SeriesInstanceUID,
  collection_id
FROM
  bigquery-public-data.idc_current.dicom_all
WHERE
  Modality = "MR" AND collection_tumorLocation = "Liver"

# note the use of this new operator that makes the query
# return just the first one of the matching rows
LIMIT
  1
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,SeriesInstanceUID,collection_id
0,1.3.6.1.4.1.14519.5.2.1.3344.4008.195410669603...,tcga_lihc


The result of this query is the _unique identifier_ for a DICOM series that meets the selection criteria.

## Selecting studies that contain series of the specific type

Another common use case for searching the data is identifying images that are accompanied by annotations of findings in those images. A significant portion of images in IDC has segementations of the structures of interest, which were created manually or using automated segmentation tools. Without going into details, such segmentations are stored in DICOM Segmentation (SEG modality) or Radiotherapy Structure Set (RTSS modality) objects.

Segmentations for a given image series within a study will be stored in a separate series. We can select studies that contain both images and segmentations using the query below. To help better summarize the result, we will list all of the modalities encountered in the given study.

In [9]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
WITH
  temp_result AS (
  SELECT
    StudyInstanceUID,
    STRING_AGG(DISTINCT(collection_id)) AS collection_id,
    STRING_AGG(DISTINCT(collection_tumorLocation)) AS collection_tumorLocation,
    ARRAY_AGG(DISTINCT(Modality)) AS modalities
  FROM
    bigquery-public-data.idc_current.dicom_all
  GROUP BY
    StudyInstanceUID)
SELECT
  StudyInstanceUID,
  ARRAY_TO_STRING(modalities,",") AS modalities,
  collection_id
FROM
  temp_result
WHERE
  "CT" IN UNNEST(modalities)
  AND "SEG" IN UNNEST(modalities)
  AND collection_tumorLocation = "Lung"
LIMIT
  1
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,StudyInstanceUID,modalities,collection_id
0,1.3.6.1.4.1.32722.99.99.3684003251586029311743...,"CT,SEG,SR,RTSTRUCT",nsclc_radiomics


This query introduces a couple of more advanced concepts:
* we use `WITH` operator to define an intermediate query that writes the result into `temp_result` table, which is then queried
* we capture all of the distinct values of `Modality` into an _array_ `modalities`, since we want to check for presence of both MR and SEG modalities in the study.

We will work with those advanced features more in the subsequent tutorials. For now, you can experiment with this query by adjusting the parameters to select studies that contain combinations of certain modalities (for example, are there imaging studies of liver cancer that contain either MR or CT images and segmentations?). This is probably the most challenging exercise from this tutorial - [BigQuery SQL workspace](https://console.cloud.google.com/bigquery) should be handy to work on it!

## Summary

After completing this tutorial, you hopefully:
* developed basic understanding of the IDC image metadata and its organization
* learned about BigQuery as the tool for searching IDC metadata
* are motivated to start experimenting with the SQL interface to select subsets of IDC data at different levels of data model (collection, patient, study, series)

If you have any questions about this tutorial, or about searching IDC metadata, please send us an email to support@canceridc.dev or posting your question on [IDC User forum](https://discourse.cancer.dev)!

This tutorial barely scratches the surface of what you can do with BigQuery SQL. If you are interested in a comprehensive tutorial about BigQuery SQL, check out this ["Intro to SQL" course on Kaggle](https://www.kaggle.com/learn/intro-to-sql)!

## Bonus

Remember, we mentioned in the beginning that you could search for the following?

* _"files corresponding to CT images of female patients that are accompanied by annotations of lung tumors that are larger than 1500 mm^3 in volume"_.

Here's the query that does that!

In [10]:
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
  # _"files corresponding to CT images of female patients that are accompanied by annotations of lung tumors that are larger than 10 ml in volume"_.
WITH

  female_ct_studies AS (
  SELECT
    DISTINCT(StudyInstanceUID),
    collection_id
  FROM
    bigquery-public-data.idc_current.dicom_all
  WHERE
    Modality = "CT"
    AND PatientSex = "F" ),

  nodules_larger_than AS (
  SELECT
    DISTINCT(SOPInstanceUID)
  FROM
    bigquery-public-data.idc_current.quantitative_measurements
  WHERE
    quantity.CodeMeaning = "Volume"
    AND value > 1500
    AND units.CodeMeaning = "cubic millimeter"
    AND finding.CodeMeaning = "Nodule" ),

  studies_for_nodules AS (
  SELECT
    StudyInstanceUID,
    collection_id
  FROM
    bigquery-public-data.idc_current.dicom_all AS dicom_all
  JOIN
    nodules_larger_than
  ON
    dicom_all.SOPInstanceUID = nodules_larger_than.SOPInstanceUID )

SELECT
  studies_for_nodules.StudyInstanceUID,
  studies_for_nodules.collection_id
FROM
  studies_for_nodules
JOIN
  female_ct_studies
ON
  studies_for_nodules.StudyInstanceUID = female_ct_studies.StudyInstanceUID
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

selection_df

,StudyInstanceUID,collection_id
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.159154526664...,lidc_idri
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.888517498954...,lidc_idri
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.908741193082...,lidc_idri
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.140071422310...,lidc_idri
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.888517498954...,lidc_idri
...,...,...
180,1.3.6.1.4.1.14519.5.2.1.6279.6001.196064403160...,lidc_idri
181,1.3.6.1.4.1.14519.5.2.1.6279.6001.554408346761...,lidc_idri
182,1.3.6.1.4.1.14519.5.2.1.6279.6001.274709137328...,lidc_idri
183,1.3.6.1.4.1.14519.5.2.1.6279.6001.196064403160...,lidc_idri


## Support

You can contact IDC support by sending email to support@canceridc.dev or posting your question on [IDC User forum](https://discourse.canceridc.dev).

## Acknowledgments

Imaging Data Commons has been funded in whole or in part with Federal funds from the National Cancer Institute, National Institutes of Health, under Task Order No. HHSN26110071 under Contract No. HHSN261201500003l.

If you use IDC in your research, please cite the following publication:

> Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S., Aerts, H. J. W. L., Homeyer, A., Lewis, R., Akbarzadeh, A., Bontempi, D., Clifford, W., Herrmann, M. D., Höfener, H., Octaviano, I., Osborne, C., Paquette, S., Petts, J., Punzo, D., Reyes, M., Schacherer, D. P., Tian, M., White, G., Ziegler, E., Shmulevich, I., Pihl, T., Wagner, U., Farahani, K. & Kikinis, R. NCI Imaging Data Commons. Cancer Res. 81, 4188–4193 (2021). http://dx.doi.org/10.1158/0008-5472.CAN-21-0950